### Required Libraries

In [49]:
# beautifulsoup4      4.9.3
# fake-useragent      0.1.11
# helium              3.0.5
# ipython             7.22.0
# matplotlib          3.3.4
# pandas              1.2.4
# Pillow              8.2.0
# requests            2.25.1
# tqdm                4.59.0

In [21]:
from helium import *
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm as tq
import time
import requests
import os
from fake_useragent import UserAgent

In [22]:
df_category = pd.read_csv('sports_rec_dataset.csv')
df_category

,product_name,product_link_name,link,major_category_name,major_category_link
0,TREADMILLS,TREADMILLS,https://www.canadiantire.ca/en/sports-rec/fitn...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
1,ELLIPTICAL MACHINES,ELLIPTICAL MACHINES,https://www.canadiantire.ca/en/sports-rec/fitn...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
2,EXERCISE BIKES,EXERCISE BIKES,https://www.canadiantire.ca/en/sports-rec/fitn...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
3,JUMP ROPES & SPEED ROPES,JUMP ROPES & SPEED ROPES,https://www.canadiantire.ca/en/sports-rec/fitn...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
4,FITNESS TRAMPOLINES & REBOUNDERS,FITNESS TRAMPOLINES & REBOUNDERS,https://www.canadiantire.ca/en/sports-rec/fitn...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
5,BOXING & MMA EQUIPMENT,BOXING & MMA EQUIPMENT,https://www.canadiantire.ca/en/sports-rec/fitn...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
6,SPORTS SUPPLEMENTS & SHAKERS,SPORTS SUPPLEMENTS & SHAKERS,https://www.canadiantire.ca/en/sports-rec/fitn...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
7,BIKES,BIKES,https://www.canadiantire.ca/en/sports-rec/bike...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
8,BIKE BASKETS & REAR RACKS,BIKE BASKETS & REAR RACKS,https://www.canadiantire.ca/en/sports-rec/bike...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
9,WATER BOTTLES & HOLDERS,WATER BOTTLES & HOLDERS,https://www.canadiantire.ca/en/sports-rec/bike...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....


In [26]:
df_category.shape

(43, 5)

In [27]:
df_category[13:16]

,product_name,product_link_name,link,major_category_name,major_category_link
13,BIKE LOCKS,BIKE LOCKS,https://www.canadiantire.ca/en/sports-rec/bike...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
14,HELMETS,HELMETS,https://www.canadiantire.ca/en/sports-rec/bike...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
15,BIKE BELLS & HORNS,BIKE BELLS & HORNS,https://www.canadiantire.ca/en/sports-rec/bike...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....


In [28]:
def update_link(link):
    split = link.split('?') 
    complement = 'https:'
    updated_link = complement + split[0]
    return updated_link

In [31]:
from selenium.webdriver.firefox.options import Options
options = Options()

ua = UserAgent()

import random
randomlist = []
for i in range(0,5):
    n = random.randint(20, 70)
    randomlist.append(n/10)

df_list = []
links_not_fetched = []
for i in tq(range(20, len(df_category))):
    print(i)
    product_name = df_category['product_name'][i]
    class_name = df_category['product_link_name'][i]
    print(df_category['link'][i])
    url = df_category['link'][i]+'?'
    
    print('Start fetching process for {}!'.format(class_name))
    
    if url[-3:] != '_no':
        number_of_pages = 5 
        url_list = [url + 'page={}'.format(i) for i in range(1, number_of_pages+1)]
    else:
        url_list = [url[:-3]]
    
    soup_list = []
    count = 0
    for page in tq(url_list):
        start = time.time()
        user_agent = ua.firefox
        options.add_argument('--user-agent="{}"'.format(user_agent))
        browser = start_firefox(page, headless=True, options=options)
        time.sleep(5)
        press(ESCAPE)
        time.sleep(8)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        checker = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        if len(checker) > 0:
            soup_list.append(soup)
            print('Page fetched: {}.'.format(page))
        elif (len(checker)==0) & (count < 2):
            count += 1
            print('Page not fetched: {}.'.format(page))
        else:
            print('Counter: ', count)
            print('Page not fetched: {}.'.format(page))
            links_not_fetched.append(page)
            print("No more pages avaiable")
            kill_browser()
            sleeping = random.choice(randomlist)
            time.sleep(sleeping)  
            end = time.time()
            time_elapsed = round(end - start, 1)
            print('Time elapsed: {}s'.format(time_elapsed))
            print('\n')
            try:
                kill_browser()
                print('Browser closed.')
            except:
                print('There were no browser open.')
                pass
            break

        try:
            kill_browser()
            print('Browser closed.')
        except:
            print('There were no browser open.')
            pass

        sleeping = random.choice(randomlist)
        time.sleep(sleeping)  
        end = time.time()
        time_elapsed = round(end - start, 1)
        print('Time elapsed: {}s'.format(time_elapsed))
    print('Fetching process for {} finished!'.format(class_name))
    print('Creating dataframe.')
    
    image_link_list = []
    original_name_list = []

    for soup in soup_list: 
        products_list = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        for product in products_list:
            image = product.find('img')
            link = image['src']
            link = update_link(link)
            original_name = image['alt']
            image_link_list.append(link)
            original_name_list.append(original_name)

    df_temp = pd.DataFrame({'original_name': original_name_list, 'image_link': image_link_list})
    df_temp['class_name'] = class_name
    df_temp['product_name'] = product_name
    df_list.append(df_temp)
    print('Dataframe for {} created.\n\n\n\n'.format(class_name))

print('Creating final dataset.')
df_final = pd.concat(df_list)
df_final.drop_duplicates(subset='image_link', inplace=True)
df_final.reset_index(inplace=True)
del df_final['index']
print('Process finished. Printing final dataset.')
df_final

  0%|          | 0/5 [00:00<?, ?it/s]

20
https://www.canadiantire.ca/en/sports-rec/camping/tents-shelters/pop-up-shelters-canopies.html?adlocation=LITs_Camping_2018_Shelters_LIT_2_WOODS_en
Start fetching process for POP-UP TENTS, SHELTERS & CANOPIES!
Page fetched: https://www.canadiantire.ca/en/sports-rec/camping/tents-shelters/pop-up-shelters-canopies.html?adlocation=LITs_Camping_2018_Shelters_LIT_2_WOODS_en?page=1.
Browser closed.



 20%|██        | 1/5 [00:55<03:40, 55.02s/it]

Time elapsed: 55.0s
Page fetched: https://www.canadiantire.ca/en/sports-rec/camping/tents-shelters/pop-up-shelters-canopies.html?adlocation=LITs_Camping_2018_Shelters_LIT_2_WOODS_en?page=2.
Browser closed.



 40%|████      | 2/5 [02:11<03:23, 67.92s/it]

...


 80%|████████  | 4/5 [03:05<00:46, 46.80s/it]

Time elapsed: 48.6s
Page fetched: https://www.canadiantire.ca/en/sports-rec/camping/trekking-poles-hiking-sticks.html?adlocation=HTML_LIT_TrekkingPolesHikingSticks_CampingDLP_en?page=5.
Browser closed.



100%|██████████| 23/23 [1:46:16<00:00, 277.23s/it]

Time elapsed: 65.2s
Fetching process for WALKING STICKS finished!
Creating dataframe.
Dataframe for WALKING STICKS created.




Creating final dataset.
Process finished. Printing final dataset.


,original_name,image_link,class_name,product_name
0,"Outbound Easy-Up Shelter, 10-ft x 10-ft",https://canadiantire.scene7.com/is/image/Canad...,"POP-UP TENTS, SHELTERS & CANOPIES","POP-UP TENTS, SHELTERS & CANOPIES"
1,Impact Canopy Traditional Pop-Up Instant Canop...,https://canadiantire.scene7.com/is/image/Canad...,"POP-UP TENTS, SHELTERS & CANOPIES","POP-UP TENTS, SHELTERS & CANOPIES"
2,Lightspeed Bahia Quick Draw Shelter,https://canadiantire.scene7.com/is/image/Canad...,"POP-UP TENTS, SHELTERS & CANOPIES","POP-UP TENTS, SHELTERS & CANOPIES"
3,Outbound OASIS Sun Shelter,https://canadiantire.scene7.com/is/image/Canad...,"POP-UP TENTS, SHELTERS & CANOPIES","POP-UP TENTS, SHELTERS & CANOPIES"
4,"Woods™ LODGE Camp Screen Shelter, 11-ft x 9-ft",https://canadiantire.scene7.com/is/image/Canad...,"POP-UP TENTS, SHELTERS & CANOPIES","POP-UP TENTS, SHELTERS & CANOPIES"
...,...,...,...,...
684,Black Diamond Explorer 3 Trekking Poles,https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS
685,Woods™ Exposure Trekking Pole,https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS
686,"Woods™ Folding Hiking Trekking Poles, Black",https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS
687,Woods™ Carbon Trekking Poles,https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS


In [32]:
df_final.to_csv('sports_rec.csv', index=False)

In [34]:
df_temp = []
for i in range(1,5):
    df_temp.append(pd.read_csv('sports_rec_{}.csv'.format(i)))
df = pd.concat(df_temp)
df.drop_duplicates(subset='image_link', inplace=True)
df.reset_index(inplace=True)
del df['index']
df

,original_name,image_link,class_name,product_name
0,NordicTrack C1750 Treadmill,https://canadiantire.scene7.com/is/image/Canad...,TREADMILLS,TREADMILLS
1,ProForm Sport 7.0C Treadmill,https://canadiantire.scene7.com/is/image/Canad...,TREADMILLS,TREADMILLS
2,HealthRider H70T Treadmill,https://canadiantire.scene7.com/is/image/Canad...,TREADMILLS,TREADMILLS
3,NordicTrack C1000 Treadmill,https://canadiantire.scene7.com/is/image/Canad...,TREADMILLS,TREADMILLS
4,NordicTrack C960i FlexSelect™ Treadmill,https://canadiantire.scene7.com/is/image/Canad...,TREADMILLS,TREADMILLS
...,...,...,...,...
1281,Black Diamond Explorer 3 Trekking Poles,https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS
1282,Woods™ Exposure Trekking Pole,https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS
1283,"Woods™ Folding Hiking Trekking Poles, Black",https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS
1284,Woods™ Carbon Trekking Poles,https://canadiantire.scene7.com/is/image/Canad...,WALKING STICKS,WALKING STICKS


In [35]:
# len(df_list)
# print('Creating final dataset.')
# df_final = pd.concat(df_list)
# df_final.drop_duplicates(subset='image_link', inplace=True)
# df_final.reset_index(inplace=True)
# del df_final['index']
# print('Process finished. Printing final dataset.')
df.to_csv('sports_rec.csv', index=False)

# Saving images

In [57]:
df_automotive = pd.read_csv('loan-a-tool.csv')
df_automotive['link'][0]

'https://www.canadiantire.ca/en/automotive/loan-a-tool/general-service-pullers.html?'

In [45]:
def get_full_link(link, image_number):    
    complement = '?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700'
    link = link[:-1]
    full_link = "{}{}{}".format(link, image_number, complement)
    return full_link

In [53]:
import os
total_images = 0
start = time.time()
links_not_fetched = []
for i in range(len(df_automotive)):
    product_name = df_automotive['product_name'][i]
    new_name = "{}_{}".format(product_name, i)
    original_name = df_automotive['original_name'][i]
    section = 'auto_parts_section'
    directory = 'files/automotive'
    link = df_automotive['image_link'][i]
    
    try:
        os.makedirs('{}/{}/{}'.format(directory, section, product_name))
    except OSError:
        pass
    
    print("\n\n Fetching images of: ", original_name, '\n')
    count = 0
    for i in range(1, 11):
        full_link = get_full_link(link, i)
        print('Link: ', full_link)
        file_name = '{}/{}/{}/{}_photo_{}.jpg'.format(directory, section, product_name, new_name, i)
        img = requests.get(full_link)      
    
        if len(img.content) < 7000:
            print('No more product images.')
            print('Number of images saved: ', count)
            break
        else:
            count += 1
            total_images += 1
            print('Saving product image.')
            with open(file_name, 'wb') as f:
                f.write(img.content)
                time.sleep(0.5)
    if count == 0:
        links_not_fetched.append(full_link)
        print('\n\n', '########### \n', 'No image in the link: ', full_link, '\n #########','\n\n')
        
end = time.time()
time_elapsed = round(end - start, 1)
print('Time elapsed: {}s'.format(time_elapsed))
print('\n')
print('Total number of images: ', total_images)       



 Fetching images of:  Littelfuse North American ATO Fuse Assortment, 6-pk 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201553_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201553_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse ATO Fuse Kit, 40-pc 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201590_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201590_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse Mini Fuse Assortment, 8-pk 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201506_1?defaultImage=image_na_EN&fmt=jpg&fit=con

No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse AGC Glass Fuse Assortment, 15-pk 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201670_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201670_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse North American Glass Fuse Assortment, 6-pk 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201550_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201550_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse ATO In-Line Fuseholder 

Link:  https://canadiantire.scene7.com/is/image/C

No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse Blade Fuse Tech, 21-pk 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201651_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201651_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse Tri-Puller Fuse Puller 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0200016_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0200016_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse OEM Chrysler/Dodge/Jeep Emergency Fuse Kit 

Link:  https://canadiantire.scene7.com/is/image/CanadianTir

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201536_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201536_5?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  4


 Fetching images of:  Littelfuse Mini Fuse, 25-pk 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201622_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201622_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse SFE 4A-9A Glass Fuse Assortment, 5-pk 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201587_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
L

No more product images.
Number of images saved:  1


 Fetching images of:  Littelfuse Emergency ATO Diagnostic Kit, 7-pc 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201557_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0201557_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  BWD Multi-Purpose Connector 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/1200037_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/1200037_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/1200037_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link

...

No more product images.
Number of images saved:  2


 Fetching images of:  NGK AB-8 Standard Spark Plug 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0183902_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0183902_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0183902_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  2


 Fetching images of:  Champion 506 D6 Spark Plug 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0183159_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0183159_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images sav

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0251099_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0251099_5?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  4


 Fetching images of:  Innova 3030RS OBD2 Code Reader 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0251094_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0251094_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0251094_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0251094_4?defaultImage=image_na_EN&fmt=jp

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0172200_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Fel-Pro Valve Cover Grommet Set 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0177220_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0177220_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Head Gasket 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0172000_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0172000_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fe

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/1172518_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Walker Exhaust Gasket, 31735 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0173210_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0173210_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  Fel-Pro Crankshaft Seal Set 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0174000_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0174000_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching im

FileNotFoundError: [Errno 2] No such file or directory: 'files/automotive/auto_parts_section/Tachometer/Oil pressure gauges/Tachometer/Oil pressure gauges_953_photo_1.jpg'

In [54]:
links_not_fetched

['https://canadiantire.scene7.com/is/image/CanadianTire/image_na_E1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700',
 'https://canadiantire.scene7.com/is/image/CanadianTire/0196010_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700',
 'https://canadiantire.scene7.com/is/image/CanadianTire/0196403_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700',
 'https://canadiantire.scene7.com/is/image/CanadianTire/0417555_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700',
 'https://canadiantire.scene7.com/is/image/CanadianTire/1128344_V1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700',
 'https://canadiantire.scene7.com/is/image/CanadianTire/1128382_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700',
 'https://canadiantire.scene7.com/is/image/CanadianTire/1272617_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700',
 'https://canadiantire.scene7.com/is/image/CanadianTire/0210500_1?

# Converting images to binary - Do not run this part!!!!

In [170]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [208]:
folder_names = df_automotive['product_name'].unique()
folder_names

array(['TIRE', 'WHEEL', 'WHEEL LUG NUTS', 'TIRE WRENCHES', 'TIRE VALVES',
       'WHEEL HUB RINGS', 'TIRE PRESSURE MONITORING SYSTEMS',
       'TIRE, WHEEL & RIM CLEANERS', 'TRACTION AIDS',
       'SPLASH & MUD FLAPS', 'TIRE INFLATORS & COMPRESSORS',
       'TIRE PRESSURE GAUGES', 'WHEEL COVERS'], dtype=object)

In [254]:
from os import walk

df_list = []
folder_names = df_automotive['product_name'].unique()
for i in tq(range(len(folder_names))):
    
    folder = folder_names[i]
    section = 'tires_and_wheels_section'
    directory = 'files/automotive'

    path = '{}/{}/{}'.format(directory, section, folder)
    _, _, file_names = next(walk(path))

    list_im2arr = []
    name_list = []
    
    print('Converting files in folder: ', folder)
    
    for i in tq(range(len(file_names))):
        file_name = file_names[i]
        full_path = '{}/{}'.format(path, file_name)
        im = Image.open(full_path)
        im2arr = np.array(im) # im2arr.shape: height x width x channel
        list_im2arr.append(im2arr)
        name_list.append(folder)
    df_temp = pd.DataFrame({'product_name': name_list, 'image_array': list_im2arr})
    df_list.append(df_temp)
df = pd.concat(df_list)
df.reset_index(inplace=True)
del df['index']

  0%|          | 0/1142 [00:00<?, ?it/s]

Converting files in folder:  TIRE



  3%|▎         | 9/316 [00:00<00:03, 87.72it/s]

Converting files in folder:  WHEEL



 21%|██        | 14/68 [00:00<00:00, 128.18it/s]

...


100%|██████████| 24/24 [00:00<00:00, 121.83it/s]

Converting files in folder:  TIRE PRESSURE GAUGES



 17%|█▋        | 10/58 [00:00<00:00, 99.00it/s]

Converting files in folder:  WHEEL COVERS



100%|██████████| 13/13 [00:33<00:00,  2.55s/it]


In [227]:
# df.to_csv('tires_and_wheels.csv', index=False)
df.to_json('tires_and_wheels.json')


KeyboardInterrupt



In [224]:
path = 'https://raw.githubusercontent.com/jonatasaguiar/hello-world/main/tires_and_wheels.csv'
path = 'tires_and_wheels.csv'
df_test = pd.read_csv(path)

In [228]:
df_test.iloc[2,1]

'[[[255 255 255]\n  [255 255 255]\n  [255 255 255]\n  ...\n  [ 38  57 149]\n  [ 38  57 149]\n  [ 38  57 149]]\n\n [[255 255 255]\n  [255 255 255]\n  [255 255 255]\n  ...\n  [ 38  57 149]\n  [ 38  57 149]\n  [ 38  57 149]]\n\n [[255 255 255]\n  [255 255 255]\n  [255 255 255]\n  ...\n  [ 38  57 149]\n  [ 38  57 149]\n  [ 38  57 149]]\n\n ...\n\n [[255 255 255]\n  [255 255 255]\n  [255 255 255]\n  ...\n  [ 42  43  45]\n  [ 38  39  41]\n  [ 43  44  46]]\n\n [[255 255 255]\n  [255 255 255]\n  [255 255 255]\n  ...\n  [ 46  47  49]\n  [ 39  40  42]\n  [ 44  45  47]]\n\n [[255 255 255]\n  [255 255 255]\n  [255 255 255]\n  ...\n  [ 48  49  51]\n  [ 40  41  43]\n  [ 44  45  47]]]'

In [229]:
# for i in range(1000,1015):
#     plt.imshow(df_test['image_array'][i])
#     plt.show()

In [232]:
df = pd.DataFrame(['[0., 1., 2., 3.]', '[1., 2., 3., 4.]'], columns=['Word Vector'])
df

,Word Vector
0,"[0., 1., 2., 3.]"
1,"[1., 2., 3., 4.]"


In [234]:
type(df['Word Vector'][0])

str

In [235]:
df['Word Vector'][0] # '[0., 1., 2., 3.]'

df['Word Vector'] = df['Word Vector'].apply(lambda x: np.array(eval(x)), 0)
df['Word Vector'][0] # array([0., 1., 2., 3.])
df

,Word Vector
0,"[0.0, 1.0, 2.0, 3.0]"
1,"[1.0, 2.0, 3.0, 4.0]"


In [236]:
type(df['Word Vector'][0])

numpy.ndarray

In [239]:
df_test_2 = df_test.head()
df_test_2

,product_name,image_array
0,TIRE,[[[255 255 255]\n [255 255 255]\n [255 255 2...
1,TIRE,[[[255 255 255]\n [255 255 255]\n [255 255 2...
2,TIRE,[[[255 255 255]\n [255 255 255]\n [255 255 2...
3,TIRE,[[[255 255 255]\n [255 255 255]\n [255 255 2...
4,TIRE,[[[255 255 255]\n [255 255 255]\n [255 255 2...


In [245]:
df_test_2['new_run'] = df_test_2['image_array'].apply(lambda x: x.replace('\n', ''))

C:\ProgramData\Anaconda3\envs\web_scraper\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [259]:
df[:100].to_pickle('test.pkl')

In [250]:
df_test_2['new_run_2'] = df_test_2['new_run'].apply(eval).apply(np.array)

SyntaxError: invalid syntax (<string>, line 1)

In [257]:
df.shape

(1805, 2)

In [ ]:
df_test = pd.read_pickle('test.pkl')

In [265]:
from PIL import Image
import os
import numpy as np

path_to_files = "files/automotive/tires_and_wheels_section/SPLASH & MUD FLAPS/"    
vectorized_images = []

for _, file in enumerate(os.listdir(path_to_files)):
    image = Image.open(path_to_files + file)
    image_array = np.array(image)
    vectorized_images.append(image_array)        
# save as DataX or any other name. But the same element name is to be used while loading it back. 
np.savez("mnistlikedataset.npz",DataX=vectorized_images) 

In [282]:
np.array([{'product_name':'test'}, vectorized_images[1]])

C:\ProgramData\Anaconda3\envs\web_scraper\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


array([{'product_name': 'test'}, array([[[115, 114, 119],
                                         [119, 118, 123],
                                         [121, 120, 125],
                                         ...,
                                         [ 28,  25,  34],
                                         [ 28,  25,  34],
                                         [ 28,  25,  34]],

                                        [[120, 121, 125],
                                         [121, 122, 126],
                                         [122, 123, 127],
                                         ...,
                                         [ 28,  25,  34],
                                         [ 28,  25,  34],
                                         [ 28,  25,  34]],

                                        [[123, 126, 133],
                                         [121, 124, 131],
                                         [118, 121, 128],
                                  

In [283]:
np.save('test_numpy', vectorized_images)

C:\ProgramData\Anaconda3\envs\web_scraper\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [285]:
np.load('test_numpy.npy', allow_pickle=True)

array([array([[[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              ...,

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
        

In [294]:
url = 'https://mylambton-my.sharepoint.com/:u:/g/personal/c0790419_mylambton_ca/ETYE-AfBjIdBq-R4ssYS-UEBPw7h_R8vWsBBuwO9ctLm6Q?e=Z2mZPt'

import requests
import io

response = requests.get(url)
response.raise_for_status()
# data = np.load(io.BytesIO(response.content), allow_pickle=True)
# # np.load(url,  allow_pickle=True)
# data
response.content

HTTPError: 403 Client Error: Forbidden for url: https://mylambton-my.sharepoint.com/personal/c0790419_mylambton_ca/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fc0790419_mylambton_ca%2FDocuments%2Ftest_numpy.npy&parent=%2Fpersonal%2Fc0790419_mylambton_ca%2FDocuments&originalPath=aHR0cHM6Ly9teWxhbWJ0b24tbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvYzA3OTA0MTlfbXlsYW1idG9uX2NhL0VUWUUtQWZCaklkQnEtUjRzc1lTLVVFQlB3N2hfUjh2V3NCQnV3TzljdExtNlE_cnRpbWU9Wi1fdWZZUW0yVWc

In [269]:
np.load('images.npy')

ValueError: Object arrays cannot be loaded when allow_pickle=False